In [133]:
import os
import cv2
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchinfo import summary

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [65]:
def transformer(train):
    transform = []
    transform.append(transforms.PILToTensor())
    transform.append(transforms.ConvertImageDtype(torch.float))
    transform.append(transforms.Resize(150,150))
    if train:
        transform.append(transforms.RandomRotation(20, fill=0))
        transform.append(transforms.RandomHorizontalFlip(0.5))
    
    return transforms.Compose(transform)

In [195]:
class Custom_Dataset(Dataset):
    def __init__(self, img_path, transformer):
        
        self.img_path = img_path
        self.transformer = transformer
        self.file_names = os.listdir(self.img_path)
        self.all_image_paths = []
        
        
#         self.classes = []

        for i in range(len(self.file_names)):
            self.all_image_paths.extend(glob.glob(self.img_path+self.file_names[i]+'/*jpg'))
#             for j in range(len(glob.glob(self.img_path+self.file_names[i]+'/*jpg'))):
#                 self.classes.append(i)
        self.idx_to_class = {i:j for i, j in enumerate(self.file_names)}
        self.class_to_idx = {value:key for key,value in idx_to_class.items()}
                
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, idx):
        
        image_path = self.all_image_paths
        img = Image.open(image_path).convert('RGB')
        label = {i:j for i, j in enumerate(self.file_names)}
        
        if self.transformer is not None:
            img = self.transformer(img)
            
        return img, label

In [156]:
def layer_maker(cfg):
    in_channel = 32
    layer = []
    
    for v in cfg:
        t, out_channel, n, stride = v
        
        for i in range(n):
            
            if i != 0:
                stride = 1
                
            conv_pointwise1 = nn.Conv2d(in_channels=in_channel, 
                                        out_channels=in_channel*t,
                                        kernel_size=1,
                                        stride=1, 
                                        padding=0)
            
            conv_depthwise = nn.Conv2d(in_channels=in_channel*t,
                                      out_channels=in_channel*t,
                                      kernel_size=3,
                                      groups=in_channel*t,
                                      stride=stride,
                                      padding=1)
            
            conv_pointwise2 = nn.Conv2d(in_channels=in_channel*t, 
                                        out_channels=out_channel,
                                        kernel_size=1,
                                        stride=1, 
                                        padding=0)
            
            layer.append([conv_pointwise1, nn.BatchNorm2d(in_channel*t), nn.ReLU6(),
                         conv_depthwise, nn.BatchNorm2d(in_channel*t), nn.ReLU6(),
                         conv_pointwise2, nn.BatchNorm2d(out_channel)])
            
            in_channel = out_channel
            
    return layer

In [163]:
cfg = {'MobileNetV2':[(1,16,1,1),
                     (6,24,2,2),
                     (6,32,3,2),
                     (6,64,4,2),
                     (6,96,3,1),
                     (6,160,3,2),
                      (6,320,1,1)]}
bottleneck = layer_maker(cfg['MobileNetV2'])
bottleneck[0]

[Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1)),
 BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU6(),
 Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32),
 BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU6(),
 Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1)),
 BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)]

In [175]:
class Mobilenet(nn.Module):
    def __init__(self, features, stride_list, num_classes = 6):
        super(Mobilenet, self).__init__()
        
        self.layer1 = nn.Sequential(nn.Conv2d(3,32,kernel_size=3, stride=2, padding=1),
                                   nn.BatchNorm2d(32))
        
        self.bottle_neck = []
        
        for layer in features:
            self.bottle_neck.append(nn.Sequential(*layer).cpu())
            
        self.layer2 = nn.Sequential(nn.Conv2d(320, 1280,kernel_size=1, stride=1, padding=0),
                                   nn.BatchNorm2d(1280),
                                   nn.AvgPool2d(5))
        
        self.fc = nn.Linear(1280, num_classes)
        
    def forward(self, x):
        
        out = self.layer1(x)
        
        for i in range(len(self.bottle_neck)):
            new = self.bottle_neck[i](out)
            if stride_list[i] == 1:
                if out.shape[1] != new.shape[1]:
                    out = Downsample(out.shape[1], new.shape[1]).cpu()(out)
                new += out
            out = new
            
        out = self.layer2(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out
    
    
class Downsample(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(Downsample, self).__init__()
        self.layer = nn.Sequential(nn.Conv2d(in_channel, out_channel, kernel_size=1),
                                  nn.BatchNorm2d(out_channel))
        
    def forward(self, x):
        out = self.layer(x)
        return out
    
    

In [176]:
stride_list = [1,
              2,1,
              2,1,1,
              2,1,1,1,
              1,1,1,
              2,1,1,
              1]

In [177]:
model = Mobilenet(bottleneck, stride_list, num_classes=6).to(device)
model

Mobilenet(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv2d(320, 1280, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): AvgPool2d(kernel_size=5, stride=5, padding=0)
  )
  (fc): Linear(in_features=1280, out_features=6, bias=True)
)

In [178]:
t_img = torch.Tensor(2,3,150,150).to(device)
out_t_img = model(t_img)
print(out_t_img.shape)
print(out_t_img)

torch.Size([2, 6])
tensor([[ 0.0165, -0.0211, -0.0446, -0.1229, -0.0763,  0.0045],
        [-0.0330,  0.0517, -0.0099,  0.0818,  0.0219, -0.0091]],
       grad_fn=<AddmmBackward0>)


In [179]:
learning_rate = 0.01
training_epochs = 100
batch_size = 16

In [180]:
a = Image.open('./dataset/archive/seg_train/seg_train/forest/10007.jpg')

In [181]:
t = transforms.Compose([transforms.PILToTensor(),
                       transforms.ConvertImageDtype(torch.float)])
img = t(a)

In [81]:
img.size()

torch.Size([3, 150, 150])

In [83]:
con1 = nn.Conv2d(3,32,kernel_size=3, stride=2, padding=1)
con2 = nn.Conv2d(32,32,kernel_size=3, stride=2, padding=1)
img1 = con1(img)
img1 = con2(img1)
img1 = con2(img1)
img1 = con2(img1)
img1 = con2(img1)
img1.size()

[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


torch.Size([32, 5, 5])

In [196]:
train_dataset = Custom_Dataset(img_path='./dataset/archive/seg_train/seg_train/',transformer=transformer)

In [197]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                               batch_size=2, 
                                               shuffle=True)

In [210]:
images,targets = next(iter(train_dataloader))
# images = list(image for image in images)
# targets = [{k: v for k, v in t.items()} for t in targets]
# targets

AttributeError: 'list' object has no attribute 'read'

In [194]:
class Custom_Dataset(Dataset):
    def __init__(self, img_path, transformer):
        
        self.img_path = img_path
        self.transformer = transformer
        self.file_names = os.listdir(img_path)
        self.all_image_paths = []
        
        
#         self.classes = []

        for i in range(len(self.file_names)):
            self.all_image_paths.extend(glob.glob(self.img_path+self.file_names[i]+'/*jpg'))
#             for j in range(len(glob.glob(self.img_path+self.file_names[i]+'/*jpg'))):
#                 self.classes.append(i)
        self.idx_to_class = {i:j for i, j in enumerate(self.file_names)}
        self.class_to_idx = {value:key for key,value in idx_to_class.items()}
                
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, idx):
        
        image_path = self.all_image_paths
        img = Image.open(image_path).convert('RGB')
        label = {i:j for i, j in enumerate(self.file_names)}
        
        if self.transformer is not None:
            img = self.transformer(img)
            
        return img, label

In [200]:
image_path = './dataset/archive/seg_train/seg_train/'
file_name = os.listdir(image_path)

In [206]:
aip = []
for i in range(len(file_name)):
    aip.extend(glob.glob(image_path+file_name[1]+'/*jpg'))

In [209]:
aip

['./dataset/archive/seg_train/seg_train/buildings/12536.jpg',
 './dataset/archive/seg_train/seg_train/buildings/16750.jpg',
 './dataset/archive/seg_train/seg_train/buildings/16988.jpg',
 './dataset/archive/seg_train/seg_train/buildings/4571.jpg',
 './dataset/archive/seg_train/seg_train/buildings/12244.jpg',
 './dataset/archive/seg_train/seg_train/buildings/2114.jpg',
 './dataset/archive/seg_train/seg_train/buildings/18769.jpg',
 './dataset/archive/seg_train/seg_train/buildings/6414.jpg',
 './dataset/archive/seg_train/seg_train/buildings/2882.jpg',
 './dataset/archive/seg_train/seg_train/buildings/11003.jpg',
 './dataset/archive/seg_train/seg_train/buildings/8363.jpg',
 './dataset/archive/seg_train/seg_train/buildings/18755.jpg',
 './dataset/archive/seg_train/seg_train/buildings/16793.jpg',
 './dataset/archive/seg_train/seg_train/buildings/14184.jpg',
 './dataset/archive/seg_train/seg_train/buildings/13827.jpg',
 './dataset/archive/seg_train/seg_train/buildings/7939.jpg',
 './dataset/ar